In [147]:
import numpy as np

In [3]:
data = open('data/day19.txt', 'r').read()

In [7]:
example = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

In [21]:
def parse_data(data):
    workflows, ratings = data.split('\n\n')
    
    parsed_workflows = {}
    for workflow in workflows.split('\n'):
        name, rules = workflow.split('{')
        rules = [rule for rule in rules[:-1].split(',')]
        parsed_workflows[name] = rules
    
    parsed_ratings = []
    for rating in ratings.split('\n'):
        category_ratings = rating[1:-1].split(',')
        parsed_ratings.append({category_rating.split('=')[0]: int(category_rating.split('=')[1]) for category_rating in category_ratings})
        
    return parsed_workflows, parsed_ratings

In [20]:
def check_passes(check, rating):
    category, operator, value = check[0], check[1], int(check[2:])
    category_rating = rating[category]
    
    return category_rating > value if operator == '>' else category_rating < value    

In [22]:
def is_accepted(workflows, rating):
    dest = 'in'
    while dest not in ['R', 'A']:
        rules = workflows[dest]
        for rule in rules:
            if ':' in rule:
                check, dest = rule.split(':')
                if check_passes(check, rating):
                    break
            else:
                dest = rule
                break
    
    return dest == 'A'

In [31]:
def part1(data):
    workflows, ratings = parse_data(data)
    total_accepted_rating_sum = 0
    for rating in ratings:
        if is_accepted(workflows, rating):
            total_accepted_rating_sum += sum(rating.values())
    
    return total_accepted_rating_sum
part1(data)

487623

In [87]:
def create_counter_rule(rule):
    check = rule.split(':')[0].replace(rule, '')
    if len(check) > 0:
        if check[1] == '<':
            return check.replace('<', '>=')
        else:
            return check.replace('>', '<=')
    else:
        return check

In [107]:
def add_counter_statements(workflows):
    new_workflows = {}
    for name, workflow in workflows.items():
        counter_rules = [create_counter_rule(rule) for rule in workflow]
        full_rules = [workflow[0]]
        for i, rule in enumerate(workflow[1:]):
            full_prior_rules = '|'.join(counter_rules[:i+1])
            full_rule = '|'.join([full_prior_rules, rule]) if ':' in rule else ':'.join([full_prior_rules, rule])
            full_rules.append(full_rule)
        new_workflows[name] = full_rules
    
    return new_workflows

In [126]:
def get_accepted_paths(workflows, source_rule, path):
    if ':' in source_rule:
        check, dest = source_rule.split(':')
        new_path = path + [check]
    else:
        dest = source_rule
        new_path = path
       
    if dest == 'A':
        return '|'.join(new_path)
    elif dest == 'R':
        return None
    else:
        dest_rules = workflows[dest]   
        accepted_paths = []
        for dest_rule in dest_rules:
            accepted_path = get_accepted_paths(workflows, dest_rule, new_path)
            if accepted_path is not None:
                accepted_paths.append(accepted_path)
        return ';'.join(accepted_paths) if len(accepted_paths) > 0 else None

In [212]:
def parse_rule_into_interval(rule):
    operator = re.findall('(>=|<=|>|<)', rule)[0]
    value = int(re.findall('\d+', rule)[0])
    
    match operator:
        case '>=':
            return (value, 4000)
        case '<=':
            return (1, value)
        case '>':
            return (value + 1, 4000)
        case '<':
            return (1, value - 1)

In [216]:
def calculate_num_distinct_combinations(accepted_paths):
    num_distinct_combinations = 0
    for path in accepted_paths:
        # Get category intervals
        category_intervals = {}
        for rule in path.split('|'):
            category = rule[0]
            interval = parse_rule_into_interval(rule)
            if category in category_intervals:
                category_intervals[category].append(interval)
            else:
                category_intervals[category] = [interval]

        # Find the intersection of each categories intervals
        path_allowed_category_values = {}
        for category in ['x', 'm', 'a', 's']:
            intervals = category_intervals.get(category, [(1, 4000)])
            min_bound = max([interval_min for (interval_min, interval_max) in intervals])
            max_bound = min([interval_max for (interval_min, interval_max) in intervals])
            path_allowed_category_values[category] = (min_bound, max_bound)

        num_distinct_combinations += np.prod([(range_max - range_min + 1) 
                                              for (range_min, range_max) in path_allowed_category_values.values()], dtype='int64')
    return num_distinct_combinations      

In [217]:
def part2(data):
    workflows, ratings = parse_data(data)
    full_workflows = add_counter_statements(workflows)
    accepted_paths = get_accepted_paths(full_workflows, 'in', []).split(';')
    num_distinct_combinations = calculate_num_distinct_combinations(accepted_paths)
    
    return num_distinct_combinations

In [219]:
part2(data)

113550238315130